In [ ]:
import ray
ray.init()

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
import pandas_bokeh
import market_cluster as mc
import polygon_ds as pds
import polygon_s3 as ps3
import flow_backfill as fbf
import ray_symbol_details as rsd

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

Install CUDA and cudamat (for python) to enable GPU speedups.


In [ ]:
# rsd.symbol_details_ray(symbols=['SPY', 'QQQ'])

In [ ]:
# ps3.remove_symbol(symbol='market', tick_type='daily')

In [ ]:
# fbf.run_backfill(symbols=['market'], tick_type='daily', start_date='2016-01-01')

In [2]:
start_date = '2018-10-24'
end_date = '2020-11-24'
symbol = 'market'
tick_type='daily'

In [5]:
df = mc.market_cluster_workflow(start_date, end_date)

4502494
-0.32428916062964214
-0.40957078454740864
-0.4373749304274475


In [ ]:
corex4000, full_df = mc.corex_fit(df, n_hidden=4000)

Linear CorEx with 4000 latent factors


In [91]:
pd.DataFrame(corex.transform(X)).corr().median().median()

0.0012913812326018105

In [63]:
pd.Series(corex.tcs).describe(percentiles=[.25,.75,.9,.99,.999])

count   100.000000
mean      4.116702
std      19.546955
min       0.187553
25%       0.291637
50%       0.385027
75%       0.867807
90%       7.186859
99%     110.293967
99.9%   157.723368
max     162.993301
dtype: float64

In [96]:
(pd.Series(corex.tcs)/sum(corex.tcs))[0:20]

0    0.351952
1    0.284141
2    0.025706
3    0.023657
4    0.023423
5    0.020460
6    0.019697
7    0.016781
8    0.016319
9    0.011220
10   0.009874
11   0.008349
12   0.008241
13   0.006850
14   0.006709
15   0.006524
16   0.005566
17   0.004925
18   0.004876
19   0.004551
dtype: float32

In [137]:
len(X.columns)

4816

In [165]:
sym_clust = pd.DataFrame(list(zip(X.columns, corex.clusters()))).rename(columns={0: 'symbol', 1: 'cluster'})

full_df = pd.merge(sym_clust, pd.Series(data=corex.tcs, name='tcs'), left_on='cluster', right_index=True)

In [210]:
full_df[full_df.symbol=='SPY']

,symbol,cluster,tcs
4016,SPY,6,52.581718


In [211]:
full_df[full_df.cluster==6]

,symbol,cluster,tcs
55,ACWF,6,52.581718
56,ACWI,6,52.581718
60,ADBE,6,52.581718
215,AMZN,6,52.581718
286,ARKK,6,52.581718
...,...,...,...
4737,XLK,6,52.581718
4766,XSW,6,52.581718
4767,XT,6,52.581718
4770,XWEB,6,52.581718


In [170]:
full_df.sort_values('tcs', ascending=False)[0:20]

,symbol,cluster,tcs
4493,VGT,0,939.526672
3530,QQQ,0,939.526672
2887,MTUM,0,939.526672
2199,IGM,0,939.526672
1754,FV,0,939.526672
1736,FTEC,0,939.526672
1550,FBGX,0,939.526672
2450,KBE,1,758.507629
688,CATY,1,758.507629
1748,FTXO,1,758.507629


In [ ]:
import corex_gaussianize as g

norm = g.Gaussianize(strategy='lambert')

In [ ]:
gld = df[df.symbol=='GLD']

In [ ]:
norm.fit(df[df.symbol=='GLD'].close)

In [ ]:
df.symbol.describe()

In [ ]:
import h2o
from h2o.estimators import H2OGeneralizedLowRankEstimator
h2o.init()

# Import the USArrests dataset into H2O:
arrestsH2O = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/pca_test/USArrests.csv")

# Split the dataset into a train and valid set:
train, valid = arrestsH2O.split_frame(ratios=[.8], seed=1234)

# Build and train the model:
glrm_model = H2OGeneralizedLowRankEstimator(k=4,
                                            loss="Huber",
                                            # loss_by_col={'cat_var': 'Categorical'},
                                            regularization_x="OneSparse", # None (default), Quadratic, L2, L1, NonNegative, OneSparse, UnitOneSparse, Simplex.
                                            regularization_y="OneSparse",
                                            gamma_x=0.5,
                                            gamma_y=0.5,
                                            max_iterations=1000,
                                            recover_svd=True,
                                            init="PlusPlus",
                                            transform="standardize",
                                           )
glrm_model.train(training_frame=train)

In [ ]:
### market clustering

# %time df = read_market_daily(result_path)
mdf = pd.read_parquet('data/mdf.parquet')

%time npdf, sharpe_ratios = mc.normalize_market_df(mdf)

# %time npdf_resid = colwise_linreg(npdf)
npdf_resid = pd.read_parquet('data/npdf_resid.parquet')

# %time par_cor_mat = npdf_resid.corr()
resid_par_cor_mat = pd.read_parquet('data/partial_cor_mat.parquet')

details_df = pd.read_parquet('data/details_df.parquet')

# cluster_lables = cluster_sim_matrix(similarity=resid_par_cor_mat.abs())
cluster_lables = pd.read_parquet('data/cluster_lables.parquet')

# symbol_meta = join_symbol_data(details_df, cluster_lables, sharpe_ratios, mdf)
symbol_meta = pd.read_parquet('data/symbol_meta.parquet')

# cluster_coheasion = mc.get_cluster_coheasion(sim_df=resid_par_cor_mat.abs(), symbol_meta=symbol_meta, cluster_col='cluster_n200')

In [ ]:
symbol_meta['liquidy_rank'] = symbol_meta.groupby('cluster_n200')['daily_avg_dollar_volume'].rank(method='first')

symbol_meta = symbol_meta.sort_values(['cluster_n200', 'liquidy_rank']).reset_index()

by_cluster = symbol_meta.groupby('cluster_n200')
result = []
for cluster, frame in by_cluster:
    counts = dict(frame['sector'].value_counts())
    counts.pop('', None)  # drop symbols w/o sector/industry info
    result.append(counts)

In [ ]:
cluster_sec = pd.DataFrame(result)    
cluster_sec = cluster_sec.drop(columns=['Finance'])
cluster_sec_pct = cluster_sec.apply(lambda x: 100 * x / float(x.sum()), axis=1)

cluster_sec_pct.style.background_gradient(cmap='coolwarm', axis=0)

# n += 1
n = 61
print(n)
symbol_meta[symbol_meta.cluster_n200==n].sector.value_counts()

sym_clust = symbol_meta[symbol_meta.cluster_n200==n]
sym_clust